## This is an example of how to proceed bivariate analysis for qualitative data using chi-square test, with Python, Pandas, NumPy and Matplotlib

In [597]:
# importing libs and setting default plot style
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import pandas as pd
import numpy as np
from scipy import stats

In [598]:
# Case: how users of a certain company evaluate the company service for customers - do specific companies relate (are associated) to 
# evaluation results or is the distribution of qualification results equal or randomly applied to all companies, irrespectively ?

# getting sample survey data from .csv file
satisfaction_dataframe = pd.read_csv("satisfaction.csv")
from pandas import option_context
with option_context('display.max_rows', 10):
    print(satisfaction_dataframe)

    COMPANY_NAME CUST_SATISFACTION
0              A               LOW
1              A               LOW
2              A               LOW
3              A               LOW
4              A               LOW
..           ...               ...
195            C            MEDIUM
196            C              HIGH
197            C              HIGH
198            C              HIGH
199            C              HIGH

[200 rows x 2 columns]


In [599]:
# creating a Pandas Dataframe with the sample data - a two-variable frequence distribution table, also know as contingency table or 
# cross tabulation - observed absolute frequencies
# reordering columns sequence
print("Table: OBSERVED ABSOLUTE FREQUENCIES")
observed_absolute_freq_dataframe = pd.crosstab(satisfaction_dataframe["COMPANY_NAME"], satisfaction_dataframe["CUST_SATISFACTION"], 
                                               margins=True, margins_name="TOTAL").reindex(["A","B","C", "TOTAL"])[["LOW","MEDIUM",
                                                                                                                    "HIGH","TOTAL"]]
observed_absolute_freq_dataframe

Table: OBSERVED ABSOLUTE FREQUENCIES


CUST_SATISFACTION,LOW,MEDIUM,HIGH,TOTAL
COMPANY_NAME,,,,
A,40,16,12,68
B,32,24,16,72
C,24,32,4,60
TOTAL,96,72,32,200


In [600]:
# getting individual totals for later calculating expected absolute frequencies
TOTAL_A = observed_absolute_freq_dataframe["TOTAL"]["A"]
TOTAL_B = observed_absolute_freq_dataframe["TOTAL"]["B"]
TOTAL_C = observed_absolute_freq_dataframe["TOTAL"]["C"]
TOTAL_LOW = observed_absolute_freq_dataframe["LOW"]["TOTAL"]
TOTAL_MEDIUM = observed_absolute_freq_dataframe["MEDIUM"]["TOTAL"]
TOTAL_HIGH = observed_absolute_freq_dataframe["HIGH"]["TOTAL"]
TOTAL = observed_absolute_freq_dataframe["TOTAL"]["TOTAL"]
print(f"TOTAL_A = {TOTAL_A}, TOTAL_B = {TOTAL_B}, TOTAL_C = {TOTAL_C}, TOTAL_LOW = {TOTAL_LOW}, TOTAL_MEDIUM = {TOTAL_MEDIUM}, TOTAL_HIGH = {TOTAL_HIGH}, TOTAL = {TOTAL}")

TOTAL_A = 68, TOTAL_B = 72, TOTAL_C = 60, TOTAL_LOW = 96, TOTAL_MEDIUM = 72, TOTAL_HIGH = 32, TOTAL = 200


In [601]:
# creating aux methods to reset values and set types when copying dataframe structure to another dataframe and thus avoid future warnings
def reset_dataframe_cells_values_to_nan(my_dataframe):
    for column_name in my_dataframe.columns:
        for index_name in my_dataframe.index:
            my_dataframe[column_name][index_name] = np.nan
            
def set_type_to_all_cells_at_dataframe(my_dataframe, my_type):
    for column_name in my_dataframe.columns:
        for index_name in my_dataframe.index:
            my_dataframe[column_name][index_name].astype(my_type)

# disable warnings regarding chained assigments for when resetting cell values later (no reference to the original df needed)
pd.options.mode.chained_assignment = None  # default='warn'

In [602]:
# creating an expected absolute frequencies dataframe from the observed frequencies dataframe template
# removing the totals (column and row)
# resetting values to NaN and float64 for avoiding warnings when overriding values
# calculating expected absolute frequencies and setting them up at the expected_absolute_freq_dataframe
expected_absolute_freq_dataframe = observed_absolute_freq_dataframe.copy()
if("TOTAL" in expected_absolute_freq_dataframe.columns):
    expected_absolute_freq_dataframe.drop("TOTAL",axis=1,inplace=True)    
if("TOTAL" in expected_absolute_freq_dataframe.index):
    expected_absolute_freq_dataframe.drop("TOTAL",axis=0,inplace=True)
reset_dataframe_cells_values_to_nan(expected_absolute_freq_dataframe)
set_type_to_all_cells_at_dataframe(expected_absolute_freq_dataframe, "float64")

expected_absolute_freq_dataframe["LOW"]["A"] = TOTAL_A*TOTAL_LOW/TOTAL
expected_absolute_freq_dataframe["LOW"]["B"] = TOTAL_B*TOTAL_LOW/TOTAL
expected_absolute_freq_dataframe["LOW"]["C"] = TOTAL_C*TOTAL_LOW/TOTAL
expected_absolute_freq_dataframe["MEDIUM"]["A"] = TOTAL_A*TOTAL_MEDIUM/TOTAL
expected_absolute_freq_dataframe["MEDIUM"]["B"] = TOTAL_B*TOTAL_MEDIUM/TOTAL
expected_absolute_freq_dataframe["MEDIUM"]["C"] = TOTAL_C*TOTAL_MEDIUM/TOTAL
expected_absolute_freq_dataframe["HIGH"]["A"] = TOTAL_A*TOTAL_HIGH/TOTAL
expected_absolute_freq_dataframe["HIGH"]["B"] = TOTAL_B*TOTAL_HIGH/TOTAL
expected_absolute_freq_dataframe["HIGH"]["C"] = TOTAL_C*TOTAL_HIGH/TOTAL

print("Table: EXPECTED ABSOLUTE FREQUENCIES")
expected_absolute_freq_dataframe

Table: EXPECTED ABSOLUTE FREQUENCIES


CUST_SATISFACTION,LOW,MEDIUM,HIGH
COMPANY_NAME,,,
A,32.64,24.48,10.88
B,34.56,25.92,11.52
C,28.80,21.60,9.60


In [603]:
# creating a residuals dataframe from the expected frequencies dataframe template
# calculating residuals (observed minus expected values) and setting them up to the residuals_dataframe

residuals_dataframe = expected_absolute_freq_dataframe.copy()

residuals_dataframe["LOW"]["A"] = observed_absolute_freq_dataframe["LOW"]["A"]-expected_absolute_freq_dataframe["LOW"]["A"]
residuals_dataframe["LOW"]["B"] = observed_absolute_freq_dataframe["LOW"]["B"]-expected_absolute_freq_dataframe["LOW"]["B"]
residuals_dataframe["LOW"]["C"] = observed_absolute_freq_dataframe["LOW"]["C"]-expected_absolute_freq_dataframe["LOW"]["C"]
residuals_dataframe["MEDIUM"]["A"] = observed_absolute_freq_dataframe["MEDIUM"]["A"]-expected_absolute_freq_dataframe["MEDIUM"]["A"]
residuals_dataframe["MEDIUM"]["B"] = observed_absolute_freq_dataframe["MEDIUM"]["B"]-expected_absolute_freq_dataframe["MEDIUM"]["B"]
residuals_dataframe["MEDIUM"]["C"] = observed_absolute_freq_dataframe["MEDIUM"]["C"]-expected_absolute_freq_dataframe["MEDIUM"]["C"]
residuals_dataframe["HIGH"]["A"] = observed_absolute_freq_dataframe["HIGH"]["A"]-expected_absolute_freq_dataframe["HIGH"]["A"]
residuals_dataframe["HIGH"]["B"] = observed_absolute_freq_dataframe["HIGH"]["B"]-expected_absolute_freq_dataframe["HIGH"]["B"]
residuals_dataframe["HIGH"]["C"] = observed_absolute_freq_dataframe["HIGH"]["C"]-expected_absolute_freq_dataframe["HIGH"]["C"]

print("Table: RESIDUALS FREQUENCIES (OBSERVED - EXPECTED)")
residuals_dataframe

Table: RESIDUALS FREQUENCIES (OBSERVED - EXPECTED)


CUST_SATISFACTION,LOW,MEDIUM,HIGH
COMPANY_NAME,,,
A,7.36,-8.48,1.12
B,-2.56,-1.92,4.48
C,-4.80,10.40,-5.60


In [604]:
# creating a chi-square dataframe from the residuals (or expected) dataframe template
# calculating chi-square values (residual^2/expected value) and setting them up to the chi_square_dataframe

chi_square_dataframe = residuals_dataframe.copy()

chi_square_dataframe["LOW"]["A"] = np.square(residuals_dataframe["LOW"]["A"])/expected_absolute_freq_dataframe["LOW"]["A"]
chi_square_dataframe["LOW"]["B"] = np.square(residuals_dataframe["LOW"]["B"])/expected_absolute_freq_dataframe["LOW"]["B"]
chi_square_dataframe["LOW"]["C"] = np.square(residuals_dataframe["LOW"]["C"])/expected_absolute_freq_dataframe["LOW"]["C"]
chi_square_dataframe["MEDIUM"]["A"] = np.square(residuals_dataframe["MEDIUM"]["A"])/expected_absolute_freq_dataframe["MEDIUM"]["A"]
chi_square_dataframe["MEDIUM"]["B"] = np.square(residuals_dataframe["MEDIUM"]["B"])/expected_absolute_freq_dataframe["MEDIUM"]["B"]
chi_square_dataframe["MEDIUM"]["C"] = np.square(residuals_dataframe["MEDIUM"]["C"])/expected_absolute_freq_dataframe["MEDIUM"]["C"]
chi_square_dataframe["HIGH"]["A"] = np.square(residuals_dataframe["HIGH"]["A"])/expected_absolute_freq_dataframe["HIGH"]["A"]
chi_square_dataframe["HIGH"]["B"] = np.square(residuals_dataframe["HIGH"]["B"])/expected_absolute_freq_dataframe["HIGH"]["B"]
chi_square_dataframe["HIGH"]["C"] = np.square(residuals_dataframe["HIGH"]["C"])/expected_absolute_freq_dataframe["HIGH"]["C"]

print("Table: CHI-SQUARE PARTIAL VALUES")
chi_square_dataframe

Table: CHI-SQUARE PARTIAL VALUES


CUST_SATISFACTION,LOW,MEDIUM,HIGH
COMPANY_NAME,,,
A,1.659608,2.937516,0.115294
B,0.189630,0.142222,1.742222
C,0.800000,5.007407,3.266667


In [605]:
# calculating total chi-square value (sum of all chi-square cells)
chi_square_total = 0
for index, row in chi_square_dataframe.iterrows():
    chi_square_total += row.sum()
print(f"CHI-SQUARE TOTAL VALUE = {round(chi_square_total,3)}")
round(chi_square_total,3)

CHI-SQUARE TOTAL VALUE = 15.861


15.861

In [606]:
# We now need to compare the chi-square total value or the p-value (for the significance level considered) with the critical value 
# in order to prove or refute the association between the two analysed variables... and that will include also the degrees of freedom
# of the chi-square distribution above. So let's calculate all of these...

In [607]:
# calculating degrees of freedom, which is the product of (the number of rows minus one) by (the number of columns minus one) => 
# (i-1)*(j-1) at our contingency table. We have 3 categories of each variable, so, the degrees of freedom is 2*2 = 4. These
# do not include the Total row or column
degrees_of_freedom = (satisfaction_dataframe["COMPANY_NAME"].unique().size-1)*(satisfaction_dataframe["CUST_SATISFACTION"].unique().size-1)
print(f"DEGREES OF FREEDOM = {degrees_of_freedom}")
degrees_of_freedom

DEGREES OF FREEDOM = 4


4

In [608]:
# calculating the critical value for our chi-square distribution and comparing to our total chi-square value
# the critical value depends on the significance level desired - e.g. alfa = 5% and the degrees of freedom at our distribution, in
# our case, df = 4.

critical_value = stats.chi2.ppf(1-.05, df=4)
print(f"CRITICAL VALUE = {round(critical_value,3)}")
round(critical_value,3)

CRITICAL VALUE = 9.488


9.488

In [609]:
# comparing our total chi-square value with the critical value, we can se, for this use case, that our chi-square value is way above
# the critical value, therefore, our two variables (company name and customer satisfaction) are closely related compared to a random 
# association.

print(f"chi_square_total = {round(chi_square_total,3)} >= critical_value = {round(critical_value,3)}")
print("H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !")

# H0 is rejected and H1 confirmed. The hypothesis that there is a not random association between the specific company name and the
# customer satisfaction regarding its costumer services

chi_square_total = 15.861 >= critical_value = 9.488
H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !


In [610]:
# Another way to confirm or reject H0 is to compare, instead of the total chi-square value and the critical value, the p_value with 
# the significance level, in our case, alfa is 5%. The p_value depends on the chi_square_total and the degrees of freedom.
p_value = round(1-stats.chi2.cdf(chi_square_total, degrees_of_freedom),3)
print(f"p_value {round(p_value,4)} < significance level 0.05")
print("H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !")
p_value

# Therefore, as our p_value is <<< than our significance level, that means H0 can be rejected and H1 assumed as valid. And thus, similar
# to the analysis above for the total chi-square and the critical value, the two variables in our distribution are definitely related
# in a not random fashion.

p_value 0.003 < significance level 0.05
H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !


0.003

In [611]:
# or:

In [612]:
# we could have used stats chi2_contingency() method to calculate everything above, directly, passing only the observed contingency 
# table data in a 2D-ndarray:

# taking observed_absolute_freq_dataframe data, removing totals and transforming into a 2D-ndarray
temp_table = observed_absolute_freq_dataframe.copy()
if("TOTAL" in temp_table.columns):
    temp_table.drop("TOTAL",axis=1,inplace=True)    
if("TOTAL" in temp_table.index):
    temp_table.drop("TOTAL",axis=0,inplace=True)
my_observed_absolute_freq_ndarray = temp_table.to_numpy()
my_observed_absolute_freq_ndarray

array([[40, 16, 12],
       [32, 24, 16],
       [24, 32,  4]])

In [613]:
# getting statistics directly (without the whole process above), by simply passing my_observed_absolute_freq_ndarray data
my_chi_square_total, my_p_value, my_degrees_of_freedom, my_expected_absolute_freq_ndarray = stats.chi2_contingency(my_observed_absolute_freq_ndarray)
my_alpha = 0.05
my_critical_value = stats.chi2.ppf(1-my_alpha, df=my_degrees_of_freedom)

print(f"my_expected_absolute_freq_ndarray =\n{my_expected_absolute_freq_ndarray}\n")
print(f"my_degrees_of_freedom = {my_degrees_of_freedom}")
print(f"my_alfa = {my_alpha}")
print(f"my_critical_value = {my_critical_value}")
print(f"my_chi_square_total = {my_chi_square_total}")
print(f"my_p_value = {my_p_value}\n")

# interpreting either by comparing chi-square 
print("INTERPRETATION:")

if my_chi_square_total >= my_critical_value:
    print(f"my_chi_square_total {round(my_chi_square_total,3)} >= my_critical_value {round(my_critical_value,3)}")
    print("H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !\n")
else:
    print(f"my_chi_square_total {round(my_chi_square_total,3)} < my_critical_value {round(my_critical_value,3)}")
    print("H0 NOT rejected. H1 rejected ! There is NO association between the company name and customer satisfaction !\n")

# or:

if my_p_value <= my_alpha:
    print(f"my_p_value {round(my_p_value,3)} < my_alpha {round(my_alpha,3)}")
    print("H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !")
else:
    print(f"my_p_value {round(my_p_value,3)} >= my_alpha {round(my_alpha,3)}")
    print("H0 NOT rejected. H1 rejected ! There is NO association between the company name and customer satisfaction !")

my_expected_absolute_freq_ndarray =
[[32.64 24.48 10.88]
 [34.56 25.92 11.52]
 [28.8  21.6   9.6 ]]

my_degrees_of_freedom = 4
my_alfa = 0.05
my_critical_value = 9.487729036781154
my_chi_square_total = 15.860566448801741
my_p_value = 0.0032120846981537215

INTERPRETATION:
my_chi_square_total 15.861 >= my_critical_value 9.488
H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !

my_p_value 0.003 < my_alpha 0.05
H0 rejected. H1 confirmed ! There is a not random association between the company name and customer satisfaction !
